#### Fetal Health prediction-classification analysis 

##### overview 

This analysis aims to predicts fetal health status  to allow Doctors and health care professions to act fast, in low and limited resource settings by identifying mothers with abnormal features on cardiotocogram. 

##### Problem
Child and Maternal Health is one of  critical issues facing our country, The UN expects by 2030, countries to end preventable death of Newborns and Under 5 years children. With the aim to reduce to at least as low as 25 per 1000.

Parallel to child mortality, also there is ** maternal mortality in 2017 accounted for 295000 **, and 94% of these occured in LMICs, in which most can be Prevented. 

Cardiotocogram is small cost effective tool that use ultrasound waves to assess both fetal and maternal health, allowing healthcare professionals to take action immediately 

##### Key Questions

1. what factors  most strongly influence Fetal health status ?
2. which Mothers and their babies requires immediate attention versus waiting? 



##### DATA Description 

Source : [text](https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification)
This dataset contains 2126 records of features extracted from Cardiotocogram exams, which were then classified by three expert obstetritians into 3 classes:

1. Normal : 1
2. Suspect : 2
3. Pathological : 3

###### key features: 

1. baseline value - FHR baseline (beats per minute)
2. accelerations - Number of accelerations per second
3. fetal_movement - Number of fetal movements per second
4. uterine_contractions - Number of uterine contractions per second
5. light_decelerations- Number of light decelerations per second
6. severe_decelerations- Number of severe decelerations per second
7. prolongued_decelerations- Number of prolonged decelerations per second
8. abnormal_short_term_variability - Percentage of time with abnormal short term variability
9. mean_value_of_short_term_variability  - Mean value of short term variability
10. percentage_of_time_with_abnormal_long_term_variability - Percentage of time with abnormal long term variability
11. mean_value_of_long_term_variability - Mean value of long term variability
12. histogram_width - Width of FHR histogram
13. histogram_min - Minimum (low frequency) of FHR histogram
14. histogram_max - Maximum (high frequency) of FHR histogram
15. histogram_number_of_peaks - Number of histogram peaks
16. histogram_number_of_zeroes - Number of histogram zeros
17. histogram_mode - Histogram mode
18. histogram_mean - Histogram mean
19. histogram_median - Histogram median
20. histogram_variance - Histogram variance
21. histogram_tendency - Histogram tendency





In [1]:
# import the necessary libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns



In [2]:
#import the dataframe 

df = pd.read_csv('fetal_health.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          2126 non-null   float64
 1   accelerations                                           2126 non-null   float64
 2   fetal_movement                                          2126 non-null   float64
 3   uterine_contractions                                    2126 non-null   float64
 4   light_decelerations                                     2126 non-null   float64
 5   severe_decelerations                                    2126 non-null   float64
 6   prolongued_decelerations                                2126 non-null   float64
 7   abnormal_short_term_variability                         2126 non-null   float64
 8   mean_value_of_short_term_variability  

###### Real -Life case scenario 

Metrics read on CTG : 
- baseline fetal heart rate : recorded over 10 minutes interval 

periodicity
- accelerations : > 15 beats per minutes for more than 15 seconds  , its normal
- decelerations: drop in baseline fetal heart rate from the baseline 

Variability: 
- variation from the baseline in period of 15 minutes 


the strip of CTG has square brackets that has one minute. 

#### Mapping the features to match real life scenario 



